In [ ]:
import importlib

import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

import vector

import sys

vector.register_awkward()

import coffea

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import itertools
from itertools import combinations

import nanoaod_analysis_tools as nat

import hepfile as hepfile

import plotting_utilities as putils

import time

#%load_ext autoreload

In [ ]:
importlib.reload(nat)

In [ ]:
print(f"python: {sys.version}\n")

print(f"numpy:   {np.__version__}")
print(f"uproot:  {uproot.__version__}")
print(f"awkward: {ak.__version__}")
print(f"vector:  {vector.__version__}")
print()

print(f"coffea:  {coffea.__version__}")


In [ ]:
start = time.time()

data_dir = '/home/bellis/top_data/NANOAOD/'

# Laptop
#infilename = 'small_skims_10k/TT_TToBCE_TuneCP5_BNV_2018_SMALL_10k.root'
#infilename = 'TTToHadronic_UL_2018_SMALL_100k.root'

# Beth Harmon
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'
#infilename = 'TTToSemiLeptonic_UL_2018.root' # Also desktop home computer
infilename = 'Reza_signal/nAOD_step_BNV_TT_TSUE/NAOD-00000_190.root' # Also desktop home computer, laptop

# Desktop home computer
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'

infile = uproot.open(data_dir + infilename)

nevents = infile["Events"].num_entries

print(f"# events: {nevents}")

print(f"Reading in {infilename}") 
dataset_type, mc_type, trigger, topology, year = nat.extract_dataset_type_and_trigger_from_filename(infilename) 
print(f"input file information:  dataset type: {dataset_type}   MC type: {mc_type}  trigger: {trigger}  topology: {topology}")

#print("Reading in events...")
#events = NanoEventsFactory.from_root(data_dir + infilename, schemaclass=NanoAODSchema).events()

print(f"\nTime to process {time.time() - start:0.3f} seconds")

In [ ]:
print("Processing data...") 

start = time.time()

events = NanoEventsFactory.from_root(data_dir + infilename, schemaclass=NanoAODSchema).events()

genpart = events.GenPart#[event_filters]

verbose = True
match_first = True

event_decay_chain_indices, decay_chain_indices, decay_chain_filename = \
          nat.generate_genpart_decay_chain_indices(genpart,infilename,verbose=verbose, match_first=match_first) 


print(f"time to process: {time.time()-start} seconds")

In [ ]:
decay_chain_data = np.load(decay_chain_filename, allow_pickle=False)

decay_chain_indices = decay_chain_data['decay_chain_indices']
event_decay_chain_indices = decay_chain_data['event_decay_chain_indices']

print("The event decay chain indices look like this!")
print("These are the events where we could match up all 6 partons/leptons")
print(event_decay_chain_indices)
print()
print("The invidual decay_chain indices look like this")
print("These are the indices of the GenPart objects that we matched up")
print(decay_chain_indices)
print()

nevents = len(events)
ndecay_chain_matched = event_decay_chain_indices.size

print(ndecay_chain_matched, decay_chain_indices.shape, nevents, "\n")
print(f"Decay chain matched: {ndecay_chain_matched} out of {nevents} for {100*ndecay_chain_matched/nevents:0.2f}\%")

In [ ]:
events[events.HLT.IsoMu24]


In [ ]:
print(len(event_decay_chain_indices))
#print(len(event_filters))

In [ ]:
print("Processing data...") 

start = time.time()

events = NanoEventsFactory.from_root(data_dir + infilename, schemaclass=NanoAODSchema).events()

print(f"Original number of events: {len(events)}")

events = events[event_decay_chain_indices]

print(f"After just grabbing truth-matched ones number of events: {len(events)}")

# CUTS AND THE LIKE

# Cut out some events
#max_events = 10000
#events = events[0:max_events]

min_jet_pt = 0 # GeV/c
max_jet_eta = 2.4
btag_min = 0.0

min_lep_pt = 0 # GeV/c
max_lep_eta = 2.4 # GeV/c


min_njets = 5
max_njets = 8

min_nleps = 1
max_nleps = 3

selected_jets = events.Jet[(events.Jet.pt > min_jet_pt) & abs(events.Jet.eta < max_jet_eta)]

#selected_leptons = events.Muon[(events.Muon.pt > min_lep_pt) & abs(events.Muon.eta < max_lep_eta) & \
#                               (events.Muon.mediumId == True)]
selected_leptons = events.Electron[(events.Electron.pt > min_lep_pt) & abs(events.Electron.eta < max_lep_eta) & \
                               (events.Electron.cutBased >= 1)]

########

event_filters = (ak.count(selected_leptons.pt, axis=1) >= min_nleps) & \
                (ak.count(selected_leptons.pt, axis=1) <= max_nleps)

event_filters = event_filters &  (ak.count(selected_jets.pt, axis=1) >= min_njets) & \
                                 (ak.count(selected_jets.pt, axis=1) <= max_njets)

#event_filters = event_filters &  ak.sum(selected_jets.btagDeepB > btag_min, axis=1) >= 1

# Trigger mask
year = 2018
trigger = 'SingleElectron'
event_filters_trigger = nat.trigger_mask(events['HLT'],trigger, year)
print(f"Trigger retains: {len(events[event_filters_trigger])}")

event_filters = event_filters & event_filters_trigger

events = events[event_filters]
event_decay_chain_indices = event_decay_chain_indices[event_filters]
decay_chain_indices = decay_chain_indices[event_filters]


print(f"time to process: {time.time()-start} seconds")

In [ ]:
#events[event_decay_chain_indices].fields

metphi = events.MET.phi
metpt = events.MET.pt

puppimetphi = events.PuppiMET.phi
puppimetpt = events.PuppiMET.pt

print(f"len(metphi): {len(metphi)}")

print(len(events))


##leptons = events[event_decay_chain_indices].Muon
#leptons = events[event_decay_chain_indices].Electron
#genparts = events[event_decay_chain_indices].GenPart

#leptons = events.Muon
leptons = events.Electron
genparts = events.GenPart


print(len(leptons), len(genparts))

In [ ]:
len(genparts)

g = genparts[0][decay_chain_indices][0]
print(g)
for a in g:
    print(a.pdgId)
    
print(g.fields)

In [ ]:
print(event_decay_chain_indices.size, decay_chain_indices.shape)

# Need this for the index of each of our matched_jets
# Because we already grabed the events that we were able to construct a complete
# decay chain for (using event_decay_chain_indices), for these entries we go from 0, 1, 2, ....
event_idx = np.arange(0,len(decay_chain_indices))

genparts = events.GenPart

gen_hadb = genparts[(event_idx, decay_chain_indices.transpose()[0])]
gen_hadWq1 = genparts[(event_idx, decay_chain_indices.transpose()[1])]
gen_hadWq2 = genparts[(event_idx, decay_chain_indices.transpose()[2])]
gen_bnvq1 = genparts[(event_idx, decay_chain_indices.transpose()[3])]
gen_bnvq2 = genparts[(event_idx, decay_chain_indices.transpose()[4])]
gen_bnvlep = genparts[(event_idx, decay_chain_indices.transpose()[5])]


# For example
print(len(gen_hadb))

In [ ]:
# This is getting the nearest jet or electron from the genparts objects using
# whatever algorithm coffea has developed.
# 
# This is *not* working with only our decay chain objects!!!! 
# It's actually grabbing the nearest jet or lepton for *all* the GenPart objects

#matched_jet = genparts[event_decay_chain_indices].nearest(events[event_decay_chain_indices].Jet)
matched_jet = genparts.nearest(events.Jet)

#matched_lep = genparts[event_decay_chain_indices].nearest(events[event_decay_chain_indices].Electron)
matched_lep = genparts.nearest(events.Electron)

print(len(matched_jet), len(matched_lep))

In [ ]:
# Now we grab the nearest jets/lepton for *only* our decay chain matched partons
# 
# Need this for the index of each of our matched_jets
# Because we already grabed the events that we were able to construct a complete
# decay chain for (using event_decay_chain_indices), for these entries we go from 0, 1, 2, ....
event_idx = np.arange(0,len(decay_chain_indices))

matched_hadb = matched_jet[(event_idx, decay_chain_indices.transpose()[0])]
matched_hadWq1 = matched_jet[(event_idx, decay_chain_indices.transpose()[1])]
matched_hadWq2 = matched_jet[(event_idx, decay_chain_indices.transpose()[2])]
matched_bnvq1 = matched_jet[(event_idx, decay_chain_indices.transpose()[3])]
matched_bnvq2 = matched_jet[(event_idx, decay_chain_indices.transpose()[4])]
matched_bnvlep = matched_lep[(event_idx, decay_chain_indices.transpose()[5])]

In [ ]:
# Some of the partons can't be matched to reconstructed objects. 
# Those fields would give us None


for x in [matched_hadb, matched_hadWq1, matched_hadWq2, \
          matched_bnvq1, matched_bnvq2, matched_bnvlep]:
    print(len(x.pt[x.pt==None]))

# So lets mask them out of everything

mask_None = (~ak.is_none(matched_hadb.pt)) & \
            (~ak.is_none(matched_hadWq1.pt)) & \
            (~ak.is_none(matched_hadWq2.pt)) & \
            (~ak.is_none(matched_bnvq1.pt)) & \
            (~ak.is_none(matched_bnvq1.pt)) & \
            (~ak.is_none(matched_bnvlep.pt))

print(ak.is_none(matched_bnvlep.pt))
print(mask_None)
print(len(mask_None[mask_None == True]))



# We're going to want to work with Vector object for the next section,
# so let's convert them!

# Vectors of GenPart objects
vec_gen_hadb = vector.Array(gen_hadb[mask_None])
vec_gen_hadWq1 = vector.Array(gen_hadWq1[mask_None])
vec_gen_hadWq2 = vector.Array(gen_hadWq2[mask_None])

vec_gen_bnvq1 = vector.Array(gen_bnvq1[mask_None])
vec_gen_bnvq2 = vector.Array(gen_bnvq2[mask_None])
vec_gen_bnvlep = vector.Array(gen_bnvlep[mask_None])

vec_matched_hadb = vector.Array(matched_hadb[mask_None])
vec_matched_hadWq1 = vector.Array(matched_hadWq1[mask_None])
vec_matched_hadWq2 = vector.Array(matched_hadWq2[mask_None])

vec_matched_bnvq1 = vector.Array(matched_bnvq1[mask_None])
vec_matched_bnvq2 = vector.Array(matched_bnvq2[mask_None])
vec_matched_bnvlep = vector.Array(matched_bnvlep[mask_None])

In [ ]:
# Print out a few for show!

for i in range(0,10):
    print(gen_hadb[i].pt, matched_hadb[i].pt, "   ", gen_hadb[i].eta, matched_hadb[i].eta, "    ", gen_hadb[i].phi, matched_hadb[i].phi)
    
print()
print()

icount = 0
for g,j in zip(vec_gen_hadb, vec_matched_hadb):
    print(f"{g.pt:7.3f} {j.pt:7.3f} {np.abs(g.pt - j.pt)/g.pt:7.3f}   {g.eta:7.3f} {j.eta:7.3f}     {g.phi:7.3f} {j.phi:7.3f}   {g.deltaR(j):7.4f}")
    
    if icount>20:
        break
    
    icount += 1


In [ ]:
mask = (vec_gen_bnvlep.deltaR(vec_matched_bnvlep)<0.4) & \
       (vec_gen_bnvq1.deltaR(vec_matched_bnvq1)<0.4) & \
       (vec_gen_bnvq2.deltaR(vec_matched_bnvq2)<0.4) & \
       ((vec_gen_bnvlep.pt - vec_matched_bnvlep.pt)/vec_gen_bnvlep.pt<3) & \
       ((vec_gen_bnvq1.pt - vec_matched_bnvq1.pt)/vec_gen_bnvq1.pt<3) & \
       ((vec_gen_bnvq2.pt - vec_matched_bnvq2.pt)/vec_gen_bnvq2.pt<3) & \
       (vec_gen_hadb.deltaR(vec_matched_hadb)<0.4) & \
       (vec_gen_hadWq1.deltaR(vec_matched_hadWq1)<0.4) & \
       (vec_gen_hadWq2.deltaR(vec_matched_hadWq2)<0.4) & \
       ((vec_gen_hadb.pt - vec_matched_hadb.pt)/vec_gen_hadb.pt<3) & \
       ((vec_gen_hadWq1.pt - vec_matched_hadWq1.pt)/vec_gen_hadWq1.pt<3) & \
       ((vec_gen_hadWq2.pt - vec_matched_hadWq2.pt)/vec_gen_hadWq2.pt<3)

    
gen_top = vec_matched_hadb + vec_matched_hadWq1 + vec_matched_hadWq2
#gen_top = matched_hadWq1 + matched_hadWq2

print(len(gen_top.mass))
print(len(gen_top.mass[mask]))

plt.hist(gen_top.mass[mask],bins=100,range=(0,300));
plt.plot([173,173],[0,800],'k--')

In [ ]:
x = vector.Array({"eta":vec_matched_hadb.eta,\
                  "rho":vec_matched_hadb.rho, \
                  "phi": vec_matched_hadb.phi, \
                  "mass":vec_matched_hadb.tau, \
                  "btagDeepB":vec_matched_hadb.btagDeepB} \
                 , with_name="Momentum4D")

y = vector.Array({"eta":vec_matched_hadWq1.eta,\
                  "rho":vec_matched_hadWq1.rho, \
                  "phi": vec_matched_hadWq1.phi, \
                  "mass":vec_matched_hadWq1.tau, \
                  "btagDeepB":vec_matched_hadWq1.btagDeepB} \
                 , with_name="Momentum4D")

z = vector.Array({"eta":vec_matched_hadWq2.eta,\
                  "rho":vec_matched_hadWq2.rho, \
                  "phi": vec_matched_hadWq2.phi, \
                  "mass":vec_matched_hadWq2.tau, \
                  "btagDeepB":vec_matched_hadWq2.btagDeepB} \
                 , with_name="Momentum4D")

a = vector.Array({"eta":vec_matched_bnvq1.eta,\
                  "rho":vec_matched_bnvq1.rho, \
                  "phi": vec_matched_bnvq1.phi, \
                  "mass":vec_matched_bnvq1.tau, \
                  "btagDeepB":vec_matched_bnvq1.btagDeepB} \
                 , with_name="Momentum4D")

b = vector.Array({"eta":vec_matched_bnvq2.eta,\
                  "rho":vec_matched_bnvq2.rho, \
                  "phi": vec_matched_bnvq2.phi, \
                  "mass":vec_matched_bnvq2.tau, \
                  "btagDeepB":vec_matched_bnvq2.btagDeepB} \
                 , with_name="Momentum4D")

c = vector.Array({"eta":vec_matched_bnvlep.eta,\
                  "rho":vec_matched_bnvlep.rho, \
                  "phi": vec_matched_bnvlep.phi, \
                  "mass":vec_matched_bnvlep.tau, \
                  "charge":vec_matched_bnvlep.charge} \
                 , with_name="Momentum4D")


had_variables_matched = nat.top_variables([x,y,z], decay_type='had')
results_matched_unsorted = nat.event_hypothesis([x,y,z,a,b],c, do_sort=False)
results_matched_sorted = nat.event_hypothesis([x,y,z,a,b],c, do_sort=True)

x

In [ ]:
x = vector.Array({"eta":vec_gen_hadb.eta,\
                  "rho":vec_gen_hadb.rho, \
                  "phi": vec_gen_hadb.phi, \
                  "mass":vec_gen_hadb.tau, \
                  "btagDeepB":vec_gen_hadb.pdgId} # Just for dummy purposes\
                 , with_name="Momentum4D")

y = vector.Array({"eta":vec_gen_hadWq1.eta,\
                  "rho":vec_gen_hadWq1.rho, \
                  "phi": vec_gen_hadWq1.phi, \
                  "mass":vec_gen_hadWq1.tau, \
                  "btagDeepB":vec_gen_hadWq1.pdgId} # Just for dummy purposes\                 
                 , with_name="Momentum4D")

z = vector.Array({"eta":vec_gen_hadWq2.eta,\
                  "rho":vec_gen_hadWq2.rho, \
                  "phi": vec_gen_hadWq2.phi, \
                  "mass":vec_gen_hadWq2.tau, \
                  "btagDeepB":vec_gen_hadWq2.pdgId} # Just for dummy purposes\
                 , with_name="Momentum4D")

a = vector.Array({"eta":vec_gen_bnvq1.eta,\
                  "rho":vec_gen_bnvq1.rho, \
                  "phi": vec_gen_bnvq1.phi, \
                  "mass":vec_gen_bnvq1.tau, \
                  "btagDeepB":vec_gen_bnvq1.pdgId} # Just for dummy purposes\                 
                 , with_name="Momentum4D")

b = vector.Array({"eta":vec_gen_bnvq2.eta,\
                  "rho":vec_gen_bnvq2.rho, \
                  "phi": vec_gen_bnvq2.phi, \
                  "mass":vec_gen_bnvq2.tau, \
                  "btagDeepB":vec_gen_bnvq2.pdgId} # Just for dummy purposes\
                 , with_name="Momentum4D")

c = vector.Array({"eta":vec_gen_bnvlep.eta,\
                  "rho":vec_gen_bnvlep.rho, \
                  "phi": vec_gen_bnvlep.phi, \
                  "mass":vec_gen_bnvlep.tau, \
                  "charge":vec_gen_bnvlep.pdgId} # Just for dummy purposes\
                 , with_name="Momentum4D")


#had_variables = nat.top_variables([x,y,z], decay_type='had', do_sort=False)
results_gen = nat.event_hypothesis([x,y,z,a,b],c, do_sort=False)

x

In [ ]:
start = time.time()

plt.figure(figsize=(16,28))

for values in [results_gen, results_matched_sorted]:
    

    #plt.figure(figsize=(16,12))

    for i,key in enumerate(values.keys()):
        #x = ak.flatten(values[i]).to_numpy()   
        #print(type(values[i]))
        x = values[key]
        if type(x) == ak.highlevel.Array:
            x = values[key].to_numpy()
        #print(type(x))

        #print(len(x),x)
        #print(len(x[x==x]))
        #print(key)
        x[x==-np.inf] = -999
        x[x==np.inf] = -999
        plt.subplot(16,4,i+1)
        if key.find('_m')>=0:
            plt.hist(x[x==x],bins=100,range=(0,350), density=True, alpha=0.5)
        elif key.find('_dR')>=0:
            plt.hist(x[x==x],bins=100,range=(0,6), density=True, alpha=0.5)
        elif key.find('_dTheta')>=0:
            plt.hist(x[x==x],bins=100,range=(0, 6.3), density=True, alpha=0.5)
        elif key.find('_pt')>=0:
            plt.hist(x[x==x],bins=100,range=(0, 200), density=True, alpha=0.5)

        #else:
        #    plt.hist(x[x==x],bins=100, density=True, alpha=0.5)
        else:
            plt.hist(x[x==x],bins=100, density=True, alpha=0.5)

        plt.title(key)

plt.tight_layout()
#plt.figure()
#plt.hist(np.cos(angle),bins=100);

print(f"\nTime to process {time.time() - start:0.3f} seconds")

# Write out the file

In [ ]:
results = results_matched_sorted
outtag = 'MATCHED_SORTED'

#results = results_gen
#outtag = 'GEN'


data = hepfile.initialize()

hepfile.create_group(data,'ml', counter='nml')
datasets = list(results.keys())    
hepfile.create_dataset(data,datasets, group='ml', dtype=float)

hepfile.create_dataset(data,['metphi', 'metpt', 'puppimetphi','puppimetpt'])


In [ ]:
def pack_awkward_arrays(data,events):
    for key in data['_GROUPS_'].keys():
        #print(key)

        for dataset in data['_GROUPS_'][key]:
            #print("\t" + dataset)
            if dataset=='COUNTER':
                continue

            if key == '_SINGLETONS_GROUP_':
                x = events[dataset].array().to_numpy()
                if x == 'triggerIsoMu24':
                    x = x.astype(int)
                #print(key,dataset,type(x), type(x[0]))
                data[dataset] = x
                data['_SINGLETONS_GROUP_/COUNTER'] = np.ones(len(x),dtype=int)
            elif dataset[0] == 'N':
                newkey = f"{key}/{dataset}"
                data[key] = events[dataset].array().to_numpy()            
            else:
                newkey = f"{key}_{dataset}"
                hepfile_newkey = f"{key}/{dataset}"
                #print(newkey,hepfile_newkey)
                data[hepfile_newkey] = ak.flatten(events[newkey].array()).to_numpy()

                counter_key = data['_MAP_DATASETS_TO_COUNTERS_'][hepfile_newkey]
                N = ak.num(events[newkey].array()).to_numpy()
                #print(len(N))
                data[counter_key] = N 

In [ ]:

for key in datasets:
    #print(key,key[3:])
    if type(results[key]) == ak.Array:
        data['ml/' + key] = results[key].to_numpy()
    elif type(results[key]) == np.ndarray:
        data['ml/' + key] = results[key]

        #print(results_reco[key])
ncombos = np.ones(len(results['had_top_m']), dtype=int) #ncombos_per_event
data['ml/nml'] = ncombos
data['_SINGLETONS_GROUP_/COUNTER'] = ncombos #np.ones(len(ncombos_per_event),dtype=int)

data['metphi'] = metphi.to_numpy()
data['metpt'] = metpt.to_numpy()
data['puppimetphi'] = puppimetphi.to_numpy()
data['puppimetpt'] = puppimetpt.to_numpy()

print(len(ncombos))
print(len(metphi))



In [ ]:
start = time.time()

outfilename = f"PROCESSED_{trigger}_{year}_{outtag}_{infilename.split('/')[-1].split('.root')[0]}.h5"
#outfilename = 'test.h5'
print(f"Writing to {outfilename}")
hepfile.write_to_file(outfilename, data, comp_type='gzip', comp_opts=9, verbose=True)

print(f"\nTime to write to file {time.time() - start:0.3f} seconds")

In [ ]:
start = time.time()

data,event = hepfile.load(outfilename)

print(f"\nTime to read file {time.time() - start:0.3f} seconds")

In [ ]:
keys = list(data['_LIST_OF_DATASETS_']) # Make a deep copy
keys.remove('ml')
keys.remove('ml/nml')
keys.remove('_SINGLETONS_GROUP_')
keys.remove('_SINGLETONS_GROUP_/COUNTER')

keys_to_plot = keys

In [ ]:
start = time.time()

plt.figure(figsize=(16,28))

for values in [data]:
    
    #plt.figure(figsize=(16,12))

    for i,key in enumerate(keys_to_plot):
        #x = ak.flatten(values[i]).to_numpy()   
        #print(type(values[i]))
        x = values[key]
        if type(x) == ak.highlevel.Array:
            x = values[key].to_numpy()
        #print(type(x))

        #print(len(x),x)
        #print(len(x[x==x]))
        #print(key)
        x[x==-np.inf] = -999
        x[x==np.inf] = -999
        plt.subplot(16,4,i+1)
        if key.find('_m')>=0:
            plt.hist(x[x==x],bins=100,range=(0,350), density=True, alpha=0.5)
        elif key.find('_dR')>=0:
            plt.hist(x[x==x],bins=100,range=(0,6), density=True, alpha=0.5)
        elif key.find('_dTheta')>=0:
            plt.hist(x[x==x],bins=100,range=(0, 6.3), density=True, alpha=0.5)
        elif key.find('_pt')>=0:
            plt.hist(x[x==x],bins=100,range=(0, 200), density=True, alpha=0.5)

        #else:
        #    plt.hist(x[x==x],bins=100, density=True, alpha=0.5)
        else:
            plt.hist(x[x==x],bins=100, density=True, alpha=0.5)

        plt.title(key)

plt.tight_layout()
#plt.figure()
#plt.hist(np.cos(angle),bins=100);

print(f"\nTime to process {time.time() - start:0.3f} seconds")

In [ ]:
#data_hadronic, event_hadronic = hepfile.load('ML_VARS_TTbarPowheg_Hadronic_2017_SMALL_1k.h5')